In [1]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error
import pandas.io.sql as sqlio

# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/roots/Data/Cleaned_Root_letters.xlsx")

In [4]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/roots/Data/Cleaned_Root_letters.xlsx")

# Prep Data for RootWords Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )
    

# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    
########################################
### GET data for  TEXT TO WORD table ###
########################################

# Create surah and ayah columns 
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 
df["Surah"] = surah
df["Ayah"] = ayah
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

# Generate our text to words table
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    TexttoWord.append( (row['index'], row['Ayah'], row['primary_key']))
    

# Create functions to insert root words, arabic words and text to word data   
def insert_root_words(root_words_data):
    """ Insert Root Words Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO RootWords (RootID, RootWord) VALUES (%s ,%s)"""
    counter = 0
    for row in root_words_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into rootWords table")
    return counter

def insert_arabic_text(arabic_text_data):
    """ Insert Arabic Text Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO ArabicWord (WordID, Word, RootID) VALUES (%s ,%s, %s)"""
    counter = 0
    for row in arabic_text_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into ArabicWord table")
    return counter

# Create function to get the quran text data 
def get_quran_text_df():
    """ Get the quran text dataframe """
    postgreSQL_select_Query = "select * from quran_text"
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from table using cursor.fetchall")
    sql_table_result = cursor.fetchall()
    quran_text_df = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
    return quran_text_df

def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (IndexID, AyahID, WordID) VALUES (%s, %s ,%s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter

if __name__ == '__main__':

    # Insert the root_words data 
    insert_root_words(root_data)

    # Insert the arabic words data 
    insert_arabic_text(arabic_words_data)
    
    # Get the quran text df
    quran_text = get_quran_text_df
    
    # Insert text_to_word_df
    insert_text_to_word(TexttoWord)

    # Close the DB connection 
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

1799 records inserted successfully into rootWords table
17623 records inserted successfully into ArabicWord table
77791 records inserted successfully into TextToWord table
PostgreSQL connection is closed


In [ ]:
# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [ ]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

In [ ]:
postgreSQL_select_Query = "select * from quran_text"
cursor.execute(postgreSQL_select_Query)
print("Selecting rows from table using cursor.fetchall")
sql_table_result = cursor.fetchall()

In [ ]:
sql_table_result[:10]

In [ ]:
quran_text = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
quran_text.head()
# print(df)

In [ ]:
# Close the DB connection 
cursor.close()
connection.close()

# List out all tables we have

In [ ]:
# Create another column for surah and ayat, extracted from ID
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 

df["Surah"] = surah
df["Ayah"] = ayah

df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)
df

In [ ]:
df_roots

In [3]:
arabic_words_df

,ARABIC,primary_key,RootID
0,بِسْمِ,2000,1641
1,السَّمَاءِ,2158,1641
2,وَالسَّمَاءَ,2199,1641
3,سَمَاوَاتٍ,2302,1641
4,الْأَسْمَاءَ,2325,1641
...,...,...,...
17618,مَسَدٍ,19606,268
17619,الصَّمَدُ,19607,1086
17620,كُفُوًا,19610,1359
17621,وَقَبَ,19614,936


In [ ]:
quran_text

# Create the TexttoWord table

In [ ]:
temp = quran_text

In [ ]:
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    TexttoWord.append( (row['index'], row['Ayah'], row['primary_key']))
    
def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (IndexID, AyahID, WordID) VALUES (%s, %s ,%s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter

insert_text_to_word(TexttoWord)


In [ ]:
temp2 = temp2.reset_index()
temp2

In [ ]:
# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in temp2.iterrows():
    TexttoWord.append( (row['index'], row['Ayah'], row['primary_key']))

In [ ]:
def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (IndexID, AyahID, WordID) VALUES (%s, %s ,%s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter


In [ ]:
insert_text_to_word(TexttoWord)

In [ ]:
# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [ ]:
postgreSQL_select_Query = "select * from quran_text"
cursor.execute(postgreSQL_select_Query)
print("Selecting rows from quran table using cursor.fetchall")
quran_text_full = cursor.fetchall()

print("Print each row and it's columns values")
index, surah, ayah, text = [], [], [], [] 
for row in quran_text_full:
    index.append(row[0])
    surah.append(row[1])
    ayah.append(row[2])
    text.append(row[3])
    
quran_text_df = pd.DataFrame(
    {'Index': index,
     'Surah': surah,
     'Ayah': ayah,
     'Text': text
    })
quran_text_df.head(10)

In [ ]:
postgreSQL_select_Query = "select * from ArabicWord"
cursor.execute(postgreSQL_select_Query)
print("Selecting rows from quran table using cursor.fetchall")
quran_text_full = cursor.fetchall()
print(quran_text_full[:100])

In [ ]:
# Close the DB connection 
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

# Now that we have the data... its time to prep it for our db

In [ ]:
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)
df

In [ ]:
TextToWord = df.merge(quran_text_full, on = ["Word", "Ayah" ], how = 'inner')

In [ ]:
quran_text_df

In [ ]:
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

In [ ]:
df.dtypes

In [ ]:
quran_text_df

In [ ]:
df

In [ ]:
quran_text

In [ ]:
df3  = quran_text_df.merge(df, on = ["Surah", "Ayah" ], how = 'inner')

In [ ]:
df3

In [ ]:
df4 = df3.merge(arabic_words_df, on = "ARABIC", how = 'inner')
df4fa885769a2d0

In [ ]:
df4.head(30)

In [ ]:
df5 = df4.drop_duplicates()
df5.nunique()

In [ ]:
df5["index2"] = df5["Index"]
df5 = df5.drop(["Index"], axis=1)

In [ ]:
df5.index

In [ ]:
# Prep data
# Put the data in the format the db accepts
text_to_word_data = []
for row in df5.index:
    AyahID = text_to_word_data.loc[row,"index2"]
    WordID = text_to_word_data.loc[row,"primary_key"]
    
    text_to_word_data.append((AyahID, WordID))


In [ ]:
df4 = df3.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'outer')
df4.head()

In [ ]:
df5 = df4.drop_duplicates()

In [ ]:
df5

In [ ]:
arabic_words_df

In [ ]:
AyahID = quran_text_df["Index"]
WordID = 

In [ ]:
# Prep Data for RootWords Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
# arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )

In [ ]:
arabic_words_df

In [ ]:
df_roots

In [ ]:
# Close the DB connection 
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

In [ ]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/roots/Data/Cleaned_Root_letters.xlsx")

# Prep Data for RootWords Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )

# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    
# Create functions to insert root words and arabic words data     
def insert_root_words(root_words_data):
    """ Insert Root Words Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO RootWords (RootID, RootWord) VALUES (%s ,%s)"""
    counter = 0
    for row in root_words_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into rootWords table")
    return counter

def insert_arabic_text(arabic_text_data):
    """ Insert Arabic Text Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO ArabicWord (WordID, Word, RootID) VALUES (%s ,%s, %s)"""
    counter = 0
    for row in arabic_text_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into ArabicWord table")
    return counter

if __name__ == '__main__':

    # Insert the root_words data 
    #insert_root_words(root_data)

    # Insert the arabic words data 
    #insert_arabic_text(arabic_words_data)

    # Close the DB connection 
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")